In [1]:
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd

In [2]:
iris = load_iris()

iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['label'] = iris.target
iris_df['label'].value_counts()

0    50
1    50
2    50
Name: label, dtype: int64

In [3]:
kfold = KFold(n_splits = 3)

n_iter = 0

for train_index, test_index in kfold.split(iris_df) :
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('## 교차검증 : {0}'.format(n_iter))
    print('학습 레이블 데이터 분포: \n', label_train.value_counts())
    print('검증 레이블 데이터 분포: \n', label_test.value_counts())
    

## 교차검증 : 1
학습 레이블 데이터 분포: 
 1    50
2    50
Name: label, dtype: int64
검증 레이블 데이터 분포: 
 0    50
Name: label, dtype: int64
## 교차검증 : 2
학습 레이블 데이터 분포: 
 0    50
2    50
Name: label, dtype: int64
검증 레이블 데이터 분포: 
 1    50
Name: label, dtype: int64
## 교차검증 : 3
학습 레이블 데이터 분포: 
 0    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포: 
 2    50
Name: label, dtype: int64


StratifiedKFold
- 클래스 불균형 문제가 있는 데이터 세트에 대해 교차 검증을 수행할 때 유용하게 사용될 수 있습니다. 클래스 불균형이란 한 클래스의 샘플 수가 다른 클래스의 샘플 수보다 훨씬 많은 경우를 말합니다.

- 이진 분류 문제를 생각해봅시다. 여기서 '양성' 클래스 샘플이 100개 있고, '음성' 클래스 샘플이 900개 있는 1000개의 샘플이 있다고 가정해보겠습니다. 이 경우, 데이터 세트는 '음성' 클래스 샘플이 대부분이므로 클래스 불균형이 있습니다.

- 일반적인 k-겹 교차 검증(KFold)를 사용하면, 일부 fold가 '양성' 클래스 샘플을 거의 포함하지 않을 수 있습니다. 이는 모델이 '양성' 클래스를 올바르게 학습하는 데 어려움을 줄 수 있습니다. 반면에 StratifiedKFold를 사용하면, 각 fold가 전체 데이터 세트의 클래스 분포를 대표하도록 만들 수 있습니다. 
- 즉, 각 fold는 '양성' 클래스 샘플의 약 10%와 '음성' 클래스 샘플의 약 90%를 포함하게 됩니다. 이렇게 하면 모델은 각 fold에서 '양성' 클래스와 '음성' 클래스를 모두 충분히 볼 수 있으므로, 클래스 불균형 문제를 완화할 수 있습니다.

- StratifiedKFold는 클래스 불균형 문제를 해결하는 데 도움이 될 수 있으며, 이는 모델의 성능을 개선하는 데 중요한 역할을 할 수 있습니다.

In [4]:
# stratifiedkfold 는 원본 데이터의 레이블 분포를 먼저 고려한 뒤 이 분포와 동일하게 학습 검증 데이터 세트를 분할함

from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits = 3)
n_iter = 0

for train_index, test_index in skf.split(iris_df, iris_df['label']) :
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('## 교차 검증 : {0}'.format(n_iter))
    print('학습 레이블 데이터 분포: \n', label_train.value_counts())
    print('검증 레이블 데이터 분포: \n', label_test.value_counts())

## 교차 검증 : 1
학습 레이블 데이터 분포: 
 2    34
0    33
1    33
Name: label, dtype: int64
검증 레이블 데이터 분포: 
 0    17
1    17
2    16
Name: label, dtype: int64
## 교차 검증 : 2
학습 레이블 데이터 분포: 
 1    34
0    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포: 
 0    17
2    17
1    16
Name: label, dtype: int64
## 교차 검증 : 3
학습 레이블 데이터 분포: 
 0    34
1    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포: 
 1    17
2    17
0    16
Name: label, dtype: int64


In [5]:
iris = load_iris()
features = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier(random_state = 156)


dt_clf = DecisionTreeClassifier(random_state = 156)

skfold = StratifiedKFold(n_splits = 3)
n_iter = 0
cv_accuracy = []

for train_index, test_index in skfold.split(features, label) :
    
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    
    n_iter += 1
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n#{0} 교차 검증 정확도: {1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
          .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스: {1}'.format(n_iter, test_index))
    cv_accuracy.append(accuracy)
    
print('\n## 교차 검증별 정확도: ', np.round(cv_accuracy, 4))
print('## 평균 검증 정확도: ', np.mean(cv_accuracy))


#1 교차 검증 정확도: 0.98, 학습 데이터 크기: 100, 검증 데이터 크기: 50
#1 검증 세트 인덱스: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115]

#2 교차 검증 정확도: 0.94, 학습 데이터 크기: 100, 검증 데이터 크기: 50
#2 검증 세트 인덱스: [ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132]

#3 교차 검증 정확도: 0.98, 학습 데이터 크기: 100, 검증 데이터 크기: 50
#3 검증 세트 인덱스: [ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]

## 교차 검증별 정확도:  [0.98 0.94 0.98]
## 평균 검증 정확도:  0.9666666666666667


cross_val_score( )
- Scikit-Learn의 cross_val_score() 함수는 기본적으로 회귀 모델에 대해서는 K-Fold 교차 검증 방법을, 분류 모델에 대해서는 Stratified K-Fold 교차 검증 방법을 사용합니다.

- 분류 모델에서 cross_val_score()를 사용하면, 데이터의 클래스 비율을 유지하며 데이터를 나누는 Stratified K-Fold 방식이 기본적으로 적용됩니다.

- 다만, 이는 기본 설정이며, 필요에 따라 cv 파라미터를 사용하여 다른 교차 검증 방법을 지정할 수 있습니다. 예를 들어, cv=KFold(n_splits=5) 와 같이 지정하면 5-Fold 교차 검증을 사용하게 됩니다.

In [6]:
from sklearn.model_selection import cross_val_score, cross_validate

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state = 156)

data = iris_data.data
label = iris_data.target

#성능 지표는 정확도 교차검증 세트는 3개

scores = cross_val_score(dt_clf, data, label, scoring = 'accuracy', cv = 3)
print('교차 검증별 정확도 : ', np.round(scores, 4))
print('평균 검증 정확도 : ', np.round(np.mean(scores), 4))

교차 검증별 정확도 :  [0.98 0.94 0.98]
평균 검증 정확도 :  0.9667


GridSearchCV
- GridSearchCV는 scikit-learn 라이브러리에서 제공하는 클래스로, 지정된 하이퍼파라미터의 모든 가능한 조합에 대해 교차 검증을 수행하여 최적의 하이퍼파라미터를 찾는 방법입니다.

- 하이퍼파라미터는 머신러닝 모델의 성능을 조절하는 매개변수들입니다. 예를 들어, 서포트 벡터 머신(SVM)에서는 마진의 넓이를 조절하는 C나 커널의 종류를 결정하는 kernel 등이 하이퍼파라미터가 됩니다. 이러한 하이퍼파라미터는 학습 데이터로부터 학습되는 것이 아니라, 사람이 직접 설정해주어야 합니다.

- GridSearchCV는 이러한 하이퍼파라미터를 효과적으로 설정하기 위한 도구입니다. 사용자가 지정한 범위 내에서 모든 하이퍼파라미터 조합을 시도하고, 각 조합에 대해 교차 검증을 수행하여 가장 성능이 좋은 조합을 찾습니다.

- GridSearchCV의 주요 파라미터는 다음과 같습니다:
    - estimator: 사용할 머신러닝 모델입니다.
    - param_grid: 하이퍼파라미터의 종류와 시도해 볼 값들을 사전 형태로 지정합니다.
    - scoring: 모델의 성능을 평가하는 지표를 지정합니다.
    - cv: 교차 검증을 수행할 때 데이터를 나누는 폴드의 개수를 지정합니다.
- GridSearchCV를 사용하면, 각각의 하이퍼파라미터 조합에 대해 성능을 자동으로 평가하므로, 수작업으로 각각의 조합을 시도해 볼 필요가 없습니다. 이는 시간을 절약하고, 또한 객체 지향 프로그래밍 방식을 사용하여 코드를 더욱 간결하고 이해하기 쉽게 만듭니다. 
- 하지만 GridSearchCV는 모든 조합을 시도하므로, 하이퍼파라미터의 개수나 시도해 볼 값의 개수가 많아지면 계산 비용이 크게 증가할 수 있습니다. 이러한 문제를 해결하기 위해 랜덤 서치(RandomizedSearchCV) 같은 다른 기법을 사용하기도 합니다.

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, test_size = 0.2, random_state = 121)

dtree = DecisionTreeClassifier()

parameters = {'max_depth' : [1, 2, 3], 'min_samples_split' : [2, 3]}

In [8]:
grid_dtree = GridSearchCV(dtree, param_grid = parameters, cv = 3, refit = True)

grid_dtree.fit(X_train, y_train)

scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', 'split0_test_score', 'split1_test_score', 'split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


In [9]:
print('GridSearchCV 최적 파라미터 : ', grid_dtree.best_params_)
print('GridSearchCV 최고 정확도 : {0:.4f}'.format(grid_dtree.best_score_))

GridSearchCV 최적 파라미터 :  {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도 : 0.9750


In [10]:
estimator = grid_dtree.best_estimator_

pred = estimator.predict(X_test)
print('테스트 데이터 세트 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))

테스트 데이터 세트 정확도: 0.9667


### 데이터 인코딩

- 레이블 인코딩

In [11]:
from sklearn.preprocessing import LabelEncoder

items = ['TV', '냉장고', '전자렌지', '컴퓨터', '선풍기', '믹서', '믹서']

encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)
print('인코딩 변환값 : ', labels)

인코딩 변환값 :  [0 1 4 5 3 2 2]


In [12]:
print('인코딩 클래스 : ', encoder.classes_)

인코딩 클래스 :  ['TV' '냉장고' '믹서' '선풍기' '전자렌지' '컴퓨터']


In [13]:
print('디코딩 원본값 : ', encoder.inverse_transform([0, 1, 4, 5, 3, 3, 2, 2]))

디코딩 원본값 :  ['TV' '냉장고' '전자렌지' '컴퓨터' '선풍기' '선풍기' '믹서' '믹서']


In [14]:
from sklearn.preprocessing import OneHotEncoder

items = ['TV', '냉장고', '전자렌지', '컴퓨터', '선풍기', '믹서', '믹서']

encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)
print('인코딩 변환값 : ', labels)

labels = labels.reshape(-1, 1)
print(labels)

oh_encoder = OneHotEncoder()
oh_labels = oh_encoder.fit_transform(labels)

print('원-핫 인코딩 데이터')
print(oh_labels.toarray())
print('원-핫 인코딩 데이터 차원')
print(oh_labels.shape)

인코딩 변환값 :  [0 1 4 5 3 2 2]
[[0]
 [1]
 [4]
 [5]
 [3]
 [2]
 [2]]
원-핫 인코딩 데이터
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]
원-핫 인코딩 데이터 차원
(7, 6)


In [15]:
# pandas를 이용해서 간편하게 원핫인코딩을 수행하세요.

df = pd.DataFrame({'item':['TV','냉장고','전자렌지','컴퓨터','선풍기','선풍기','믹서','믹서'] })
df = pd.get_dummies(df)
df

,item_TV,item_냉장고,item_믹서,item_선풍기,item_전자렌지,item_컴퓨터
0,1,0,0,0,0,0
1,0,1,0,0,0,0
2,0,0,0,0,1,0
3,0,0,0,0,0,1
4,0,0,0,1,0,0
5,0,0,0,1,0,0
6,0,0,1,0,0,0
7,0,0,1,0,0,0


### 피처 스케일링과 정규화

StandardScaler와 MinMaxScaler는 모두 scikit-learn 라이브러리의 데이터 전처리 도구입니다. 이들은 특성 스케일링(feature scaling)을 수행하는 데 사용되며, 이는 모든 특성이 동일한 스케일을 갖도록 변환하는 과정입니다. 특성의 스케일이 다르면 머신러닝 알고리즘의 성능에 부정적인 영향을 미칠 수 있습니다.

- StandardScaler: 이 스케일러는 특성을 표준화합니다. 특성의 평균을 0, 표준 편차를 1로 변경하여 정규 분포를 따르도록 만듭니다. 이는 각 특성의 값에서 특성의 평균을 빼고 표준 편차로 나눔으로써 수행됩니다. 표준화는 특성의 분포가 정규 분포가 아닐 때, 또는 특성 간의 스케일 차이를 제거하려 할 때 유용합니다.

- MinMaxScaler: 이 스케일러는 특성을 정규화합니다. 특성의 최소값을 0, 최대값을 1로 변경하여 모든 특성이 0과 1 사이의 값으로 변환되도록 합니다. 이는 각 특성의 값에서 특성의 최소값을 빼고, 그 결과를 특성의 범위(최대값 - 최소값)로 나눔으로써 수행됩니다. 정규화는 특성의 분포가 균등하게 퍼져 있을 때, 또는 특성의 최소값과 최대값이 명확히 정해져 있을 때 유용합니다.

이 두 가지 스케일러는 모두 데이터의 분포나 값의 범위를 변경하지만, 어떤 스케일러를 사용할지는 문제의 요구 사항과 데이터의 특성에 따라 달라집니다. 또한, 이들 스케일러를 사용할 때는 항상 훈련 데이터를 기반으로 스케일러를 학습시킨 후, 이를 훈련 데이터와 테스트 데이터에 모두 적용해야 합니다. 이렇게 해야 테스트 데이터가 훈련 데이터와 동일한 방식으로 변환되므로, 모델이 일관된 입력을 받을 수 있습니다.


In [16]:
iris = load_iris()
iris_data = iris.data
iris_df = pd.DataFrame(data = iris_data, columns = iris.feature_names)

print(iris_df.mean(), '\n')
print(iris_df.var())

sepal length (cm)    5.843333
sepal width (cm)     3.057333
petal length (cm)    3.758000
petal width (cm)     1.199333
dtype: float64 

sepal length (cm)    0.685694
sepal width (cm)     0.189979
petal length (cm)    3.116278
petal width (cm)     0.581006
dtype: float64


In [17]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(iris_df)
iris_scaled = scaler.transform(iris_df)

iris_df_scaled = pd.DataFrame(data = iris_scaled, columns = iris.feature_names)
print('평균값')
print(iris_df_scaled.mean(), '\n')
print('분산값')
print(iris_df_scaled.var())

평균값
sepal length (cm)   -1.690315e-15
sepal width (cm)    -1.842970e-15
petal length (cm)   -1.698641e-15
petal width (cm)    -1.409243e-15
dtype: float64 

분산값
sepal length (cm)    1.006711
sepal width (cm)     1.006711
petal length (cm)    1.006711
petal width (cm)     1.006711
dtype: float64


In [18]:
# minmaxscaler를 사용하여 정규화

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaler.fit(iris_df)
iris_scaled = scaler.transform(iris_df)

iris_df_scaled = pd.DataFrame(data = iris_scaled, columns = iris.feature_names)
print('최소값')
print(iris_df_scaled.min(), '\n')
print('최대값')
print(iris_df_scaled.max())

최소값
sepal length (cm)    0.0
sepal width (cm)     0.0
petal length (cm)    0.0
petal width (cm)     0.0
dtype: float64 

최대값
sepal length (cm)    1.0
sepal width (cm)     1.0
petal length (cm)    1.0
petal width (cm)     1.0
dtype: float64


스케일러를 사용할 때는 항상 훈련 데이터를 기반으로 스케일러를 학습시킨 후, 이를 훈련 데이터와 테스트 데이터에 모두 적용
- fit 메서드는 훈련 데이터에 대한 스케일링 파라미터(평균과 표준편차)를 계산하고, transform 메서드는 이 파라미터를 사용해 데이터를 실제로 변환합니다. fit 메서드는 훈련 데이터에만 적용되어야 하며, 이렇게 학습된 스케일러는 훈련 데이터와 테스트 데이터에 모두 적용

In [76]:
X, y = np.arange(20).reshape((10, 2)), range(10)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

scaler = StandardScaler()

scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

### 과제 1 와인 데이터에 대해서 아래 사항을 고려하여 모델 생성및 성능 개선을 위한 하이퍼 파라미터 튜닝 수행

- dt를 알고리즘으로 수행
- cv = 5
- param_grid = {'max_depth' : [3, 4, 5, 6], 'min_samples_split' :  [2, 3, 4]}

In [20]:
from sklearn import datasets

wine = datasets.load_wine()
X, y = wine.data, wine.target
wine.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names'])

In [21]:
print(wine.feature_names, '\n')
print(wine.target, '\n')
print(wine.target_names)

['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline'] 

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2] 

['class_0' 'class_1' 'class_2']


In [22]:
# 1

dt_clf = DecisionTreeClassifier(random_state = 121)

kfold = KFold(n_splits = 5)

n_iter = 0
cv_accuracy = []

for train_index, test_index in kfold.split(X, y) :
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    
    n_iter += 1
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n#{0} 교차 검증정확도 : {1}, 학습 데이터 크기 : {2}, 검증 데이터 크기 : {3}'
         .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스 : {1}'.format(n_iter, test_index))
    cv_accuracy.append(accuracy)

    print('\n## 교차 검증별 정확도: ', np.round(cv_accuracy, 4))
print('## 평균 검증 정확도: ', np.mean(cv_accuracy))


#1 교차 검증정확도 : 0.9167, 학습 데이터 크기 : 142, 검증 데이터 크기 : 36
#1 검증 세트 인덱스 : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35]

## 교차 검증별 정확도:  [0.9167]

#2 교차 검증정확도 : 0.8056, 학습 데이터 크기 : 142, 검증 데이터 크기 : 36
#2 검증 세트 인덱스 : [36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
 60 61 62 63 64 65 66 67 68 69 70 71]

## 교차 검증별 정확도:  [0.9167 0.8056]

#3 교차 검증정확도 : 0.8056, 학습 데이터 크기 : 142, 검증 데이터 크기 : 36
#3 검증 세트 인덱스 : [ 72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107]

## 교차 검증별 정확도:  [0.9167 0.8056 0.8056]

#4 교차 검증정확도 : 0.7429, 학습 데이터 크기 : 143, 검증 데이터 크기 : 35
#4 검증 세트 인덱스 : [108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142]

## 교차 검증별 정확도:  [0.9167 0.8056 0.8056 0.7429]

#5 교차 검증정확도 : 0.9429, 학습 데이터 크기 : 143, 검증 데이터 크기 : 35
#5 검증 세

In [23]:
# 2 - cross_val

dt = DecisionTreeClassifier(random_state = 121)

scores = cross_val_score(dt, X, y, scoring = 'accuracy', cv = 5)
print('교차 검증별 정확도 : ', np.round(scores, 4))
print('평균 검증 정확도 : ', np.round(np.mean(scores), 4))

교차 검증별 정확도 :  [0.9444 0.8056 0.9167 0.9143 0.8286]
평균 검증 정확도 :  0.8819


In [24]:
wine = datasets.load_wine()
X, y = wine.data, wine.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 121)

param_grid = {'max_depth' : [3, 4, 5, 6], 'min_samples_split' : [2, 3, 4]}

dt = DecisionTreeClassifier()

grid_dt = GridSearchCV(dt, param_grid = param_grid, cv = 5)

grid_dt.fit(X_train, y_train)

scores_df = pd.DataFrame(grid_dt.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', 'split0_test_score', 'split1_test_score', 
           'split2_test_score', 'split3_test_score', 'split4_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score
0,"{'max_depth': 3, 'min_samples_split': 2}",0.911000,8,0.96,0.92,0.92,0.88,0.875000
1,"{'max_depth': 3, 'min_samples_split': 3}",0.902667,12,0.96,0.92,0.92,0.88,0.833333
2,"{'max_depth': 3, 'min_samples_split': 4}",0.919000,2,0.96,0.92,0.92,0.92,0.875000
3,"{'max_depth': 4, 'min_samples_split': 2}",0.919000,2,0.96,0.92,0.92,0.92,0.875000
4,"{'max_depth': 4, 'min_samples_split': 3}",0.919000,2,0.96,0.92,0.92,0.92,0.875000
5,"{'max_depth': 4, 'min_samples_split': 4}",0.919000,2,0.96,0.92,0.92,0.92,0.875000
6,"{'max_depth': 5, 'min_samples_split': 2}",0.927000,1,0.96,0.92,0.92,0.96,0.875000
7,"{'max_depth': 5, 'min_samples_split': 3}",0.919000,2,0.96,0.92,0.92,0.92,0.875000
8,"{'max_depth': 5, 'min_samples_split': 4}",0.910667,9,0.96,0.92,0.92,0.92,0.833333
9,"{'max_depth': 6, 'min_samples_split': 2}",0.910667,9,0.96,0.92,0.92,0.92,0.833333


In [25]:
print(grid_dt.best_params_)
print(grid_dt.best_score_)

{'max_depth': 5, 'min_samples_split': 2}
0.9269999999999999


### 과제2 아래 동물 데이터로 원 핫 인코딩을 수행한후 데이터 프레임으로 출력하세요

In [26]:
animals = ['고양이', '개', '새', '개', '새', '새', '고양이', '고양이']

encoder = LabelEncoder()
encoder.fit(animals)
labels = encoder.transform(animals)

labels = labels.reshape(-1, 1)

oh_encoder = OneHotEncoder()
oh_labels = oh_encoder.fit_transform(labels)

print('원-핫 인코딩 데이터')
print(oh_labels.toarray())
print('원-핫 인코딩 데이터 차원')
print(oh_labels.shape)

원-핫 인코딩 데이터
[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]
원-핫 인코딩 데이터 차원
(8, 3)


In [27]:
df = pd.DataFrame({'animals' : ['고양이', '개', '새', '개', '새', '새', '고양이', '고양이']})
df = pd.get_dummies(df)
df

,animals_개,animals_고양이,animals_새
0,0,1,0
1,1,0,0
2,0,0,1
3,1,0,0
4,0,0,1
5,0,0,1
6,0,1,0
7,0,1,0


### 과제3 타이타닉 생존자 예측 모델을 아래 사항을 참조하여 개선

In [41]:
import seaborn as sns

%matplotlib inline

titanic_df = pd.read_csv('../machinerunning/data.machine/train.csv')
# titanic_df = pd.read_csv('../8.머신러닝/train.csv')


titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [42]:
print('\n ### train 데이터 정보 ### \n')
print(titanic_df.info())


 ### train 데이터 정보 ### 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [43]:
titanic_df.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.036847,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500
Age,0.036847,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067
SibSp,-0.057527,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000


In [44]:
titanic_df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [45]:
import re

titanic_df['name_title'] = titanic_df['Name'].str.extract(r', ([^\.]+).')

name_title = titanic_df.groupby('name_title')['Age'].agg('mean').round().astype(int).reset_index()


titanic_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,name_title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,Rev
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,Miss
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,Miss
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,Mr


In [46]:
null_age = titanic_df['Age'].isnull()

null_name = titanic_df.loc[null_age, 'name_title']

mean_age = name_title.set_index('name_title')['Age']

titanic_df.loc[null_age, 'Age'] = null_name.map(mean_age)

In [47]:
titanic_df['Age'] = titanic_df['Age'].astype(int)

In [48]:
title_mapping = {
    "Mr": 0,
    "Miss": 1,
    "Mrs": 2,
    "Master": 3,
    "Rev": 4,
    "Dr": 5,
    "Col": 6,
    "Mlle": 7,
    "Ms": 8,
    "Major": 9,
    "Capt": 10,
    "Sir": 11,
    "Dona": 12,
    "Jonkheer": 13,
    "the Countess": 14,
    "Don": 15,
    "Mme": 16,
    "Lady": 17
}

titanic_df["name_title"] = titanic_df["name_title"].map(title_mapping)

In [49]:
def get_category(age) :
    cat = ''
    if age <= 5 : cat = '0'
    elif age <= 12 : cat = '1'
    elif age <= 18 : cat = '2'
    elif age <= 25 : cat = '3'
    elif age <= 35 : cat = '4'
    elif age <= 60 : cat = '5'
    else : cat = '6'
        
    return cat

titanic_df['age_cat'] = titanic_df['Age'].apply(get_category).astype(int)

titanic_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,name_title,age_cat
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S,0,3
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C,2,5
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S,1,4
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S,2,4
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27,0,0,211536,13.0000,NaN,S,4,4
887,888,1,1,"Graham, Miss. Margaret Edith",female,19,0,0,112053,30.0000,B42,S,1,3
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,22,1,2,W./C. 6607,23.4500,NaN,S,1,3
889,890,1,1,"Behr, Mr. Karl Howell",male,26,0,0,111369,30.0000,C148,C,0,4


In [50]:
titanic_df['family'] = 0
titanic_df.loc[(titanic_df['SibSp'] > 0) | (titanic_df['Parch'] > 0), 'family'] = 1

titanic_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,name_title,age_cat,family
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S,0,3,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C,2,5,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S,1,4,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S,2,4,1
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27,0,0,211536,13.0000,NaN,S,4,4,0
887,888,1,1,"Graham, Miss. Margaret Edith",female,19,0,0,112053,30.0000,B42,S,1,3,0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,22,1,2,W./C. 6607,23.4500,NaN,S,1,3,1
889,890,1,1,"Behr, Mr. Karl Howell",male,26,0,0,111369,30.0000,C148,C,0,4,0


In [51]:
titanic_df['Sex'].replace({'male' : 0, 'female' : 1}, inplace = True)

In [52]:
titanic_df.drop(['PassengerId', 'Name', 'SibSp', 'Parch', 'Ticket', 'Cabin', 'Embarked'], axis = 1, inplace = True)

In [53]:
titanic_df

,Survived,Pclass,Sex,Age,Fare,name_title,age_cat,family
0,0,3,0,22,7.2500,0,3,1
1,1,1,1,38,71.2833,2,5,1
2,1,3,1,26,7.9250,1,4,0
3,1,1,1,35,53.1000,2,4,1
4,0,3,0,35,8.0500,0,4,0
...,...,...,...,...,...,...,...,...
886,0,2,0,27,13.0000,4,4,0
887,1,1,1,19,30.0000,1,3,0
888,0,3,1,22,23.4500,1,3,1
889,1,1,0,26,30.0000,0,4,0


In [54]:
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Survived    891 non-null    int64  
 1   Pclass      891 non-null    int64  
 2   Sex         891 non-null    int64  
 3   Age         891 non-null    int32  
 4   Fare        891 non-null    float64
 5   name_title  891 non-null    int64  
 6   age_cat     891 non-null    int32  
 7   family      891 non-null    int64  
dtypes: float64(1), int32(2), int64(5)
memory usage: 48.9 KB


In [55]:
# titanic_df.to_pickle('../8.머신러닝/titanic_df.pickle')
# titanic_df = pd.read_pickle('../8.머신러닝/titanic_df.pickle')

titanic_df.to_pickle('../machinerunning/data.machine/titanic_df.csv')
titanic_df = pd.read_pickle('../machinerunning/data.machine/titanic_df.csv')

In [56]:
# 과제로 주신것
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# 독립변수, 종속변수 분리
y_t_df = titanic_df['Survived'] # 종속변수
X_t_df = titanic_df.drop('Survived', axis = 1) # 독립변수

# 독립변수 정규화
# X_t_df = preprocessing.StandardScaler().fit(X_t_df).transform(X_t_df)

# 학습용 데이터와 평가용 데이터를 8:2 혹은 7:3으로 분리
X_train, X_test, y_train, y_test = train_test_split(X_t_df, y_t_df, test_size = 0.2,
                                                   random_state = 11)

print(X_train.shape)
print(X_test.shape)


(712, 7)
(179, 7)


In [75]:
# 모델 학습 및 평가
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
accuracy_rf = accuracy_score(y_test, rf_pred).round(2)

lr_model = LogisticRegression()
lr_model.fit(X_train,y_train)
lr_pred = lr_model.predict(X_test)
accuracy_lr = accuracy_score(y_test,lr_pred).round(2)

print('rf 정확도:{}, lr 정확도:{}'.format(accuracy_rf,accuracy_lr))

rf 정확도:0.87, lr 정확도:0.85


C:\Users\h\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
